# Assistant-10<br/>Knowledge Retrieval

Notes:
- As of 01/14/2024 there's still a bug with knowledge retrieval.


### Get an OpenAI client

**Note:** When configuring the OpenAI client, the API version, the model version, and endpoint are "new".

In [1]:
import oaihelper as helper

## Get an OpenAI client
client = helper.get_openai_client(api_key=helper.api_KEY,
        api_version=helper.api_version,
        azure_endpoint=helper.api_URI,)

### Load the files

In [2]:
## Files
file_benefits = helper.upload_file(client,"../data/contoso-benefits.txt")
file_401k = helper.upload_file(client,"../data/contoso-401k.txt")
file_leave = helper.upload_file(client,"../data/contoso-medical_leave.txt")

Added file:  assistant-frvSdofG8Rt6OhG63iJ8kAgb 1
Added file:  assistant-kEsQXn8Tq1w3X9RYBtzhTXW0 2
Added file:  assistant-CoeVzGt77tmzCtxIJbwl80Cv 3


### Define the tools to use in the Assistants

In [3]:
tools_list = [        
        {
            "type": "function",
            "function": {
                "name": "send_email",
                "description": "Sends an email to a recipient(s).",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "to": {
                            "type": "string",
                            "description": "The email(s) the email should be sent to."
                        },
                        "content": {
                            "type": "string",
                            "description": "The content of the email."
                        }
                    },
                    "required": ["to", "content"]
                }
            }
        },
        {"type","retrieval"}
    ]

### Define the HR Assistant

In [4]:
helper.clear_shelves()

hr_assistant = helper.create_assistant(client,
        name="HR Assistant",
        instructions="You HR support chatbot. Use your knowledge base to best respond to the employee queries. Be friendly, professional, and helpful. If you don't know the answer, please say, 'contact HR for more information.'",
        tools=[{"type": "retrieval"}],
        model=helper.gpt_deployment_name,
        file_ids=[file_benefits.id, file_401k.id, file_leave.id],)

Added assistant:  asst_b7FY5qTrSdaPERJj0pL03v8H 1


### Define the delegate to handle function calling

In [5]:
def function_calling_delegate(client, run, thread):
    print("Function Calling")
    required_actions = run.required_action.submit_tool_outputs.model_dump()
    print(required_actions)
    tool_outputs = []
    import json
    for action in required_actions["tool_calls"]:
        func_name = action['function']['name']
        arguments = json.loads(action['function']['arguments'])
        
        if func_name == "send_email":
            print("Sending email...")
            email_to = arguments['to']
            email_content = arguments['content']
            helper.send_email({'to': email_to,'content':email_content})
            
            tool_outputs.append({
                "tool_call_id": action['id'],
                "output": "Email sent"
            })
        else:
            raise ValueError(f"Unknown function: {func_name}")
        
    print("Submitting outputs back to the Assistant...")
    client.beta.threads.runs.submit_tool_outputs(
        thread_id=thread.id,
        run_id=run.id,
        tool_outputs=tool_outputs
    )

### Process a user message

In [6]:
helper.generate_response(client,hr_assistant,"Can you list some company benefits? Can you provide a summary for requesting medical leave?", "123", "Alex", function_calling_delegate)
helper.generate_response(client,hr_assistant,"Can you provide me a complete list of company benefits benefits?", "123", "Alex", function_calling_delegate)
helper.generate_response(client,hr_assistant,"What is the 401k matching percentage?", "123", "Alex", function_calling_delegate)

Creating new thread for Alex with user_id 123
Added thread:  thread_lPvkMylnfd1iw0sifHStLg2H 1
Generated message: Here is a summary of the company benefits:

- **Health Insurance**: The company covers 100% of the premiums for medical, dental, and vision plans for employees and their dependents, along with flexible spending accounts, health savings accounts, and wellness programs.
- **Retirement Savings**: There's a 401(k) plan with a 50% match of employees' contributions up to 6% of their salary. Financial education and planning services are also provided.
- **Paid Time Off**: Contoso offers generous policies for vacation, sick leave, holidays, parental leave, and sabbatical leave, encouraging employees to take time to recharge.
- **Professional Development**: Opportunities for learning new skills and career advancement are provided through tuition reimbursement, online courses, mentoring programs, and internal mobility.
- **Work-Life Balance**: Flexible work arrangements, remote work 

"The company's 401(k) matching percentage is 50% of the employee's contributions, with the match applying to up to 6% of the employee's salary. This means if an employee contributes 6% or more of their salary to their 401(k) plan, Contoso will contribute an additional 3% (which is 50% of the employee's 6%)."

### Delete all demo objects

In [7]:
helper.cleanup(client)

Deleting:  1  assistants.
AssistantDeleted(id='asst_b7FY5qTrSdaPERJj0pL03v8H', deleted=True, object='assistant.deleted')
Deleting:  1  threads.
ThreadDeleted(id='thread_lPvkMylnfd1iw0sifHStLg2H', deleted=True, object='thread.deleted')
Deleting:  3  files.
FileDeleted(id='assistant-frvSdofG8Rt6OhG63iJ8kAgb', deleted=True, object='file')
FileDeleted(id='assistant-kEsQXn8Tq1w3X9RYBtzhTXW0', deleted=True, object='file')
FileDeleted(id='assistant-CoeVzGt77tmzCtxIJbwl80Cv', deleted=True, object='file')
